<a href="https://colab.research.google.com/github/123prashanth123/Fault-Detection-System/blob/Colabs/Model_Viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

In [1]:
from IPython.display import clear_output

!pip install torchviz
!pip install hiddenlayer

clear_output()

In [2]:
import torch
from torch import nn
from torchvision import models, transforms

from torchviz import make_dot
import hiddenlayer as HL

import warnings
warnings.filterwarnings("ignore")

# Siamese

In [3]:
class SiameseNetwork(nn.Module):
    def __init__(self, IL=2048, embed=4096):
        super(SiameseNetwork, self).__init__()

        self.embedder = nn.Sequential()
        self.embedder.add_module("BN", nn.BatchNorm1d(num_features=IL, eps=1e-5))
        self.embedder.add_module("FC", nn.Linear(in_features=IL, out_features=embed))
        self.embedder.add_module("AN", nn.ReLU())

        self.classifier = nn.Sequential()
        self.classifier.add_module("BN", nn.BatchNorm1d(num_features=embed, eps=1e-5))
        self.classifier.add_module("FC", nn.Linear(in_features=embed, out_features=1))

    def getOptimizer(self, lr=1e-3, wd=0):
        return optim.Adam(self.parameters(), lr=lr, weight_decay=wd)

    def getScheduler(self, optimizer=None, patience=5, eps=1e-8):
        return optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=patience, eps=eps, verbose=True)

    def forward(self, x1, x2=None):
        if x2 is not None:
            x1 = self.embedder(x1)
            x2 = self.embedder(x2)
            x = self.classifier(torch.abs(x1 - x2))
            return x
        else:
            x = self.classifier(self.embedder(x1))
            return x

model = SiameseNetwork()
model.eval()

clear_output()

x = torch.rand(8, 2, 2048)


# TORCHVIZ
out = model(x[:, 0, :], x[:, 1, :])
make_dot(out, params=dict(list(model.named_parameters()))).render("render_from_torchviz", format="png")


# HIDDENLAYER
# Removes Constant nodes from graph.
transforms = [HL.transforms.Prune("Constant")]
graph = HL.build_graph(model, (x[:, 0, :], x[:, 1, :]), transforms=transforms)
# graph = HL.build_graph(model, (x[:, 0, :]), transforms=transforms)
graph.theme = HL.graph.THEMES["blue"].copy()
graph.save("render_from_hiddenlayer", format="png")

# Feature Extractor

In [4]:
"""
    ONNX doesn't support Adaptive Average Pool.
"""

class FE(nn.Module):
    def __init__(self):
        super(FE, self).__init__()
        self.model = models.vgg16_bn(pretrained=True, progress=True)
        self.model = nn.Sequential(*[*self.model.children()][:-2])
        self.model.add_module("Flatten", nn.Flatten())
    
    def forward(self, x):
        return self.model(x)

model = FE()
model.eval()

x = torch.rand(8, 3, 224, 224)
op = model(x)

transforms = [HL.transforms.Prune("Constant")]
graph = HL.build_graph(model, x, transforms=transforms)
graph.theme = HL.graph.THEMES["blue"].copy()
graph.save("render_from_hiddenlayer_fe", format="png")

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
